### Day 18

### Part 1:
- Numbers are in nested lists
- Add a + b = [a,b], where a and b are a number or list
- But max depth is 3, so if a = [[[c,d]]] then need to reduce [a,b] = [[[[c,d]]],b] through these rules:
    - Starting from left, find first number that satisfies one of these and apply it, then repeat
    - If a pair is at depth >3, explode it:
        - Add the left number to the number to the left (if it exists)
        - Add the right number to the number to the right (if it exists)
        - Put a zero where the pair was
    - If one number is 10 or greater, reduce it:
        - Break it into the pair [floor(x/2), ceil(x/2)]

Thoughts:
- Use strings?
    - Simple to handle looking back/forward
    - Difficult to handle numbers >10
- Binary tree?
    - Will get messy to find next/previous number

In [1]:
def fix_explode(num):
    """Fix the first number pair that needs to be exploded"""
    depth = -1
    had_issue = False
    
    for ix,l in enumerate(num[:-1]):
        if l == "[":
            depth += 1
        elif l == "]":
            depth -= 1
        elif l ==",":
            continue
            
        if depth > 3:
            # Explode
            new_num = explode(num,ix)
            had_issue = True
            break

    if had_issue:
        num = new_num
    
    return had_issue, num

def fix_split(num):
    """Fix the first number pair that needs to be exploded"""
    had_issue = False
    
    for ix,l in enumerate(num[:-1]):            
        # The number is >9 if the next part of the string is also a number
        if (num[ix] not in ",[]") and (num[ix+1] not in ",[]"):
            new_num = split(num,ix)
            had_issue = True
            break

    if had_issue:
        num = new_num
    
    return had_issue, num

def fix_num_once(num):
    """Explode or split the number once."""
    # Explode if needed
    had_issue, num = fix_explode(num)
    
    # Otherwise Split if needed
    if not had_issue:
        had_issue, num = fix_split(num)
    return had_issue, num

def fix_num(num):
    """Loop to fix a number."""
    had_issue = True
    while had_issue:
        had_issue, num = fix_num_once(num)
    return num

def explode(num,ix):
    """Apply the rules to explode a pair that has a depth that is too high"""
    left_str = num[:ix]
    # Find the numbers
    split = num[ix+1:].split("]",1)
    right_str = split[1]
    left_num, right_num = split[0].split(",")
    left_num, right_num = int(left_num), int(right_num)
    
    # Find first number from the right and add left_num
    new_left_str = ""
    found_left_num = False
    l_ix = 0
    while l_ix < len(left_str):
        l = left_str[-l_ix-1]
        
        if (l not in ",[]") and (not found_left_num):
            # Account for two digit numbers
            if left_str[-l_ix-2] not in ",[]":
                l = left_str[-l_ix-2]+l
                l_ix += 1
            l = str(int(l)+left_num)
            found_left_num = True
            
        new_left_str = l+new_left_str
        l_ix += 1

    # Find first number from the left and add right_num
    new_right_str = ""
    found_right_num = False
    l_ix = 0
    while l_ix < len(right_str):
        l = right_str[l_ix]
        
        if (l not in ",[]") and (not found_right_num):
            # Account for two digit numbers
            if right_str[l_ix+1] not in ",[]":
                l = l+right_str[l_ix+1]
                l_ix += 1

            l = str(int(l)+right_num)
            found_right_num = True
            
        new_right_str += l
        l_ix += 1

            
    new_num = new_left_str + "0" + new_right_str

    return new_num
    
def split(num,ix):
    """Split a number >10 into two numbers. Assume max two digits."""
    left_str = num[0:ix]
    to_split = int(num[ix:ix+2])
    right_str = num[ix+2:]
    
    # Split the number in two
    left_num = to_split//2
    right_num = to_split//2 + (to_split %2)
    
    # Remake the string
    new_num = left_str + f"[{left_num},{right_num}]" + right_str
    return new_num

def add(num1,num2):
    """Add two Snailfish numbers."""
    num = f"[{num1},{num2}]"
    num = fix_num(num)
    return num

def read_and_add(fname):
    """Read a file with multiple Snailfish numbers and add them all in order."""
    with open(fname, "r") as f:
        data = f.read().splitlines()
        
    num = data[0]
    for l in data[1:]:
        num = add(num,l)
        
    return num

def mag_recursive(l:list):
    """Recursively calculate the magnitude of a snailfish number (as a list)."""
    if isinstance(l[0],list):
        n1 = mag_recursive(l[0])
    else:
        n1 = l[0]
    
    if isinstance(l[1],list):
        n2 = mag_recursive(l[1])
    else:
        n2 = l[1]
    
    return 3*n1 + 2*n2
    
def magnitude(num_str):
    """Calculate magnitude of Snailfish number string."""
    # Turn it into a list
    num_list = eval(num_str)
    
    # Now be recursive
    mag = mag_recursive(num_list)

    return mag

In [2]:
# Test inputs
num = "[[[[[9,8],1],2],3],4]" # [[[[0,9],2],3],4]
num = "[7,[6,[5,[4,[3,2]]]]]" # [7,[6,[5,[7,0]]]]
num = "[[6,[5,[4,[3,2]]]],1]" # [[6,[5,[7,0]]],3]
num = "[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]" # [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]] after one iteration
# fix_num_once(num)

In [3]:
# Test inputs from a file
num = read_and_add("inputs/day18_test_input.dat")
print(num)
magnitude(num)

[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]


4140

In [4]:
# Puzzle input
num = read_and_add("inputs/day18_input.dat")
print(num)
magnitude(num)

[[[[6,6],[7,7]],[[7,7],[7,7]]],[[[7,7],[0,7]],[[8,8],[8,7]]]]


4072

### Part 2:
- Try adding all pairs of inputs and find the max magnitude

In [5]:
def find_max_mag(fname):
    with open(fname, "r") as f:
        data = f.read().splitlines()
        
    max_mag = 0
    n = len(data)
    
    # Loop over all pairs
    for ix1 in range(n):
        for ix2 in range(n):
            if ix1 != ix2:
                n1 = data[ix1]
                n2 = data[ix2]
                
                num = add(n1,n2)
                mag = magnitude(num)
                
                max_mag = max(mag,max_mag)
        
    return max_mag


In [6]:
# Test input
find_max_mag("inputs/day18_test_input.dat")

3993

In [7]:
# Puzzle input
find_max_mag("inputs/day18_input.dat")

4483